## Calcular Métricas - Tempo de Votação, Biometria, etc.

---

## Importing libraries

In [1]:
import duckdb
import pandas as pd
import time

## Importing Data

In [2]:
cursor = duckdb.connect()

Dados Brutos

In [4]:
TABLE = "read_parquet('VOTES.parquet/*/*/*/*.parquet', hive_partitioning=True)"
ZONE_GROUPS = [ (x, x+20) for x in range(0, 800, 20) ]

Adicionar TURNO e Timestamp final de Biometria

In [5]:
source = F"""
(
    SELECT 
        *,
        
        CASE event_date
            WHEN '2022-10-02' THEN 1
            WHEN '2022-10-03' THEN 1
            WHEN '2022-10-30' THEN 2
            WHEN '2022-10-31' THEN 2
            ELSE NULL
        END::INT AS turno,

        COALESCE(
            timestamp_biometria_1,
            timestamp_biometria_2,
            timestamp_biometria_3,
            timestamp_biometria_4,
            timestamp_biometria_manual
        ) AS timestamp_biometria_final,

        strftime( '%Y-%m-%d %H:', timestamp_voto_computado )
        || (EXTRACT(MINUTE FROM timestamp_voto_computado)//5)*5 + 5
        || ':00' AS timestamp_voto_computado_5min
        
    FROM 
        {TABLE}
) _
"""

In [34]:
df = cursor.execute(F"""
    SELECT 
        strftime( '%Y-%m-%d %H:', timestamp_voto_computado )
        || (EXTRACT(MINUTE FROM timestamp_voto_computado)//5)*5 + 5
        || ':00' AS timestamp_voto_computado_5min,
        * 
    FROM {source}
    WHERE quantidade_votos_computados = 1
    LIMIT 10
""").df()
df

,timestamp_voto_computado,minute,timestamp_voto_computado_5min,event_date,uf,filename,vote_id,city_code,zone_code,zone_group,...,timestamp_biometria_1,timestamp_biometria_2,timestamp_biometria_3,timestamp_biometria_4,timestamp_biometria_manual,timestamp_habilitacao_eleitor,timestamp_titulo_digitado,timestamp_voto_computado_1,turno,timestamp_biometria_final
0,2022-10-02 06:16:41,15,2022-10-02 06:20:00,2022-10-02,AC,o00407-0100700090008,5.0,01007,0009,0-100,...,2022-10-02 06:15:05,2022-10-02 06:15:22,NaT,NaT,NaT,2022-10-02 06:15:35,2022-10-02 06:15:04,2022-10-02 06:16:41,1,2022-10-02 06:15:05
1,2022-10-02 06:23:41,20,2022-10-02 06:25:00,2022-10-02,AC,o00407-0100700090008,9.0,01007,0009,0-100,...,2022-10-02 06:22:15,NaT,NaT,NaT,NaT,2022-10-02 06:22:24,2022-10-02 06:22:02,2022-10-02 06:23:41,1,2022-10-02 06:22:15
2,2022-10-02 06:33:35,30,2022-10-02 06:35:00,2022-10-02,AC,o00407-0100700090008,14.0,01007,0009,0-100,...,2022-10-02 06:31:42,2022-10-02 06:31:52,2022-10-02 06:31:59,2022-10-02 06:32:07,NaT,2022-10-02 06:32:18,2022-10-02 06:31:40,2022-10-02 06:33:35,1,2022-10-02 06:31:42
3,2022-10-02 07:18:36,15,2022-10-02 07:20:00,2022-10-02,AC,o00407-0100700090008,30.0,01007,0009,0-100,...,2022-10-02 07:17:43,NaT,NaT,NaT,NaT,2022-10-02 07:17:53,2022-10-02 07:17:40,2022-10-02 07:18:36,1,2022-10-02 07:17:43
4,2022-10-02 07:26:02,25,2022-10-02 07:30:00,2022-10-02,AC,o00407-0100700090008,37.0,01007,0009,0-100,...,2022-10-02 07:23:46,NaT,NaT,NaT,NaT,2022-10-02 07:23:56,2022-10-02 07:23:44,2022-10-02 07:26:02,1,2022-10-02 07:23:46
5,2022-10-02 08:13:43,10,2022-10-02 08:15:00,2022-10-02,AC,o00407-0100700090008,54.0,01007,0009,0-100,...,2022-10-02 08:12:17,NaT,NaT,NaT,NaT,2022-10-02 08:12:32,2022-10-02 08:12:12,2022-10-02 08:13:43,1,2022-10-02 08:12:17
6,2022-10-02 08:32:18,30,2022-10-02 08:35:00,2022-10-02,AC,o00407-0100700090008,62.0,01007,0009,0-100,...,NaT,NaT,NaT,NaT,NaT,2022-10-02 08:30:49,2022-10-02 08:30:36,2022-10-02 08:32:18,1,NaT
7,2022-10-02 08:37:25,35,2022-10-02 08:40:00,2022-10-02,AC,o00407-0100700090008,64.0,01007,0009,0-100,...,2022-10-02 08:34:26,2022-10-02 08:34:40,NaT,NaT,NaT,2022-10-02 08:35:00,2022-10-02 08:34:25,2022-10-02 08:37:25,1,2022-10-02 08:34:26
8,2022-10-02 08:38:55,35,2022-10-02 08:40:00,2022-10-02,AC,o00407-0100700090008,65.0,01007,0009,0-100,...,2022-10-02 08:38:02,NaT,NaT,NaT,NaT,2022-10-02 08:38:12,2022-10-02 08:38:00,2022-10-02 08:38:55,1,2022-10-02 08:38:02
9,2022-10-02 09:03:36,0,2022-10-02 09:5:00,2022-10-02,AC,o00407-0100700090008,75.0,01007,0009,0-100,...,2022-10-02 09:02:16,NaT,NaT,NaT,NaT,2022-10-02 09:02:43,2022-10-02 09:02:15,2022-10-02 09:03:36,1,2022-10-02 09:02:16


## Preparinga Data

Méticas no Cubo OLAP - Turno, UF, Zona, Seção.

- Número de Votos
- Número de Seções Eleitorais
- Média, Soma, q50%, q90% do Tempo total de Voto, Tempo de Biometria, Tempo Total

- Quantidade de Votos efetuados em até 30s, 1min, 1min30s, 2min, 2min30s, 3min+
- Taxa de Sucesso da Biometria em 1 tentativa, 2 tentativas, 3 tentativas, 4 tentativas, Falha
- Quantidade de Teclas Pressionadas
- Quantidade de Cargos Distintos Votados

**Definição das métricas de tempo**

In [ ]:
tempo_voto_total = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_titulo_digitado))"
tempo_voto       = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_habilitacao_eleitor))"
tempo_biometria  = "EXTRACT(EPOCH FROM (timestamp_biometria_final - timestamp_titulo_digitado))"

intervalos_tempo_segundos_votos = [0, 30, 60, 90, 120, 150, 180, 210, 300, 9999]
contagem_de_votos_em_intervalos_de_tempo = ", ".join([
    F"""
    SUM( 
        CASE WHEN 
        {tempo_voto} >= {intervalos_tempo_segundos_votos[i]} 
        AND {tempo_voto} < {intervalos_tempo_segundos_votos[i+1]}
        THEN 1 ELSE 0 END 
    ) AS votos_{intervalos_tempo_segundos_votos[i]}_{intervalos_tempo_segundos_votos[i+1]}_segundos
    """
    for i in range(0, len(intervalos_tempo_segundos_votos)-1)
])